# ENTRENADOS
## ResNet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
import time
from tqdm import tqdm



In [2]:
!cp -r /content/drive/MyDrive/SorghumWeedDataset_Classification /content/


In [3]:
DATASET_PATH = "/content/SorghumWeedDataset_Classification/"

IMG_SIZE = (224, 224)

In [6]:
print(torch.cuda.is_available())

True


In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        out += identity
        out = F.relu(out)
        return out

In [5]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet18, self).__init__()

        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1   = nn.BatchNorm2d(64)
        self.relu  = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual stages (2 blocks per stage)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels

        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [7]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(f"{DATASET_PATH}/Train", transform)
val_data   = datasets.ImageFolder(f"{DATASET_PATH}/Validate", transform)
test_data  = datasets.ImageFolder(f"{DATASET_PATH}/Test", transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando:", device)

model = ResNet18(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


Usando: cuda


In [11]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        print(f"\nEpoch {epoch+1}/{epochs}")

        progress_bar = tqdm(train_loader, desc="Training", ncols=100)

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            # Actualizar barra con métricas actuales
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{correct/total:.4f}"
            })

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")


In [15]:
train_model(model, train_loader, val_loader, epochs=10)



Epoch 1/10


Training: 100%|████████████████████████████| 95/95 [00:13<00:00,  6.85it/s, loss=0.3634, acc=0.8476]


Epoch 1/10 | Loss: 0.4592 | Acc: 0.8476

Epoch 2/10


Training: 100%|████████████████████████████| 95/95 [00:13<00:00,  6.87it/s, loss=0.1112, acc=0.9400]


Epoch 2/10 | Loss: 0.1791 | Acc: 0.9400

Epoch 3/10


...

Epoch 8/10 | Loss: 0.0850 | Acc: 0.9715

Epoch 9/10


Training: 100%|████████████████████████████| 95/95 [00:13<00:00,  6.79it/s, loss=0.3180, acc=0.9682]


Epoch 9/10 | Loss: 0.0919 | Acc: 0.9682

Epoch 10/10


Training: 100%|████████████████████████████| 95/95 [00:14<00:00,  6.72it/s, loss=0.0685, acc=0.9808]

Epoch 10/10 | Loss: 0.0609 | Acc: 0.9808


In [18]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    print("\nEvaluating...")
    progress_bar = tqdm(loader, desc="Testing", ncols=100)

    with torch.no_grad():
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = outputs.max(1)

            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{correct/total:.4f}"
            })

    accuracy = correct / total
    avg_loss = running_loss / len(loader)
    print(f"Final Accuracy: {accuracy:.4f} | Final Loss: {avg_loss:.4f}")
    return accuracy, avg_loss

In [19]:
evaluate(model, test_loader)


Evaluating...


Testing: 100%|█████████████████████████████| 14/14 [00:01<00:00, 11.94it/s, loss=0.0695, acc=0.8353]

Final Accuracy: 0.8353 | Final Loss: 0.4348


(0.8352668213457076, 0.434768194532288)